In [1]:
#import usual packages for data manipulation
import pandas as pd
import numpy as np

#import usual packages for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

#import yfinance to get stock data
import yfinance as yf

#import quantstats to get some statistics on the stock data
import quantstats as qs

#import statistical packages
import statsmodels.api as sm
import scipy.stats as scs

#import datetime to get the current date
import datetime as dt

#set the style of the plots
plt.style.use('seaborn-v0_8-whitegrid')

In [2]:
#Arvind's API key
my_key = 'DJ3QKKTFZ5J298QY'

In [4]:


#from alpha_vantage import EconIndicators
from alpha_vantage.econindicators import EconIndicators

In [37]:
#instantiate the econindicators class
ei = EconIndicators(key=my_key, output_format='pandas',indexing_type='integer')

In [38]:
#get following date: quarterly gdp percap, monthly treasury yield (10yrs), monthly unemployment rate, monthly inflation rate, monthly consumer price index
gdp_percap, _ = ei.get_real_gdp_per_capita()
treasury_yield, _ = ei.get_treasury_yield(interval='monthly',maturity='10year')
federal_funds_rate, _ = ei.get_ffr(interval='monthly')
consumer_price_index, _ = ei.get_cpi(interval='monthly')
inflation, _ = ei.get_inflation()
unemployment_rate, _ = ei.get_unemployment()

In [62]:
#loop through all dataframes above and do the following:
#1. drop the index column
#2. reset the index with drop=True
#3. convert the date column to datetime and set it as the index
#4. drop all rows corresponding to years before 2014

for df in [gdp_percap, treasury_yield, federal_funds_rate, consumer_price_index, inflation, unemployment_rate]:
    df.drop(columns='index',inplace=True)
    df.reset_index(drop=True,inplace=True)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date',inplace=True)
    df.drop(df[df.index.year < 2014].index,inplace=True)

#merge all dataframes into one
df_econindicators = pd.concat([gdp_percap, treasury_yield, federal_funds_rate, consumer_price_index, inflation, unemployment_rate],axis=1)
#rename columns
df_econindicators.columns = ['gdp_percap','treasury_yield','federal_funds_rate','consumer_price_index','inflation','unemployment_rate']
#forward fill missing values
df_econindicators = df_econindicators.ffill()
#save to csv
df_econindicators.to_csv('../data/econindicators.csv')

In [96]:
df_econindicators

,gdp_percap,treasury_yield,federal_funds_rate,consumer_price_index,inflation,unemployment_rate
date,,,,,,
2014-01-01,56345.0,2.86,0.07,233.916,1.62222297740827,6.6
2014-02-01,56345.0,2.71,0.07,234.781,1.62222297740827,6.7
2014-03-01,56345.0,2.72,0.08,236.293,1.62222297740827,6.7
2014-04-01,56968.0,2.71,0.09,237.072,1.62222297740827,6.2
2014-05-01,56968.0,2.56,0.09,237.900,1.62222297740827,6.3
...,...,...,...,...,...,...
2024-03-01,67672.0,4.21,5.33,312.332,4.11633838374488,3.8
2024-04-01,68070.0,4.54,5.33,313.548,4.11633838374488,3.9
2024-05-01,68070.0,4.48,5.33,314.069,4.11633838374488,4.0


In [3]:
#import company financials data with the following code:

from alpha_vantage.fundamentaldata import FundamentalData
fd = FundamentalData(key=my_key)

#choose a ticker
ticker = 'MSFT'

#get quarterly earnings for "ticker"
df_earnings, _ = fd.get_earnings_quarterly(ticker)

#get quarterly balance sheet for "ticker"
df_balance, _ = fd.get_balance_sheet_quarterly(ticker)

#get quarterly cash flow for "ticker"
df_cashflow, _ = fd.get_cash_flow_quarterly(ticker)

#get quarterly income statement for "ticker"
df_income, _ = fd.get_income_statement_quarterly(ticker)

In [7]:
#code to clean dataframes
for df in [df_income, df_balance, df_cashflow, df_earnings]:

    #if any dates are duplicated, keep the first one and drop the rest
    df.drop_duplicates(subset='fiscalDateEnding',keep='first',inplace=True) 

    #drop all columns with at least 10 missing values
    df.dropna(axis=1,thresh=df.shape[0]-10,inplace=True)

    #convert the 'fiscalDateEnding' column to datetime and set it as the index
    df['date'] = pd.to_datetime(df['fiscalDateEnding'])
    df.drop(columns='fiscalDateEnding',inplace=True)
    df.set_index('date',inplace=True)

    #drop all rows corresponding to years before 2014
    df.drop(df[df.index.year < 2014].index,inplace=True)

#merge dataframes into one
df_financials = pd.concat([df_income, df_balance, df_cashflow, df_earnings],axis=1,join='inner')
#drop duplicate columns
df_financials = df_financials.loc[:,~df_financials.columns.duplicated()]


In [47]:

len(df_financials.columns)

93

In [60]:
df_ratios = df_financials[['reportedEPS','estimatedEPS','surprise','surprisePercentage','ebit',
 'ebitda']].astype(float)
df_ratios['reportedDate'] = df_financials['reportedDate']
# define various financial ratios from df_financials
df_ratios['debt_to_equity'] = df_financials['totalLiabilities'].astype(float) / df_financials['totalShareholderEquity'].astype(float)
df_ratios['current_ratio'] = df_financials['totalCurrentAssets'].astype(float) / df_financials['totalCurrentLiabilities'].astype(float)
df_ratios['quick_ratio'] = (df_financials['totalCurrentAssets'].astype(float) - df_financials['inventory'].astype(float)) / df_financials['totalCurrentLiabilities'].astype(float)
df_ratios['return_on_equity'] = df_financials['netIncome'].astype(float) / df_financials['totalShareholderEquity'].astype(float)

In [59]:
df_ratios

,reportedEPS,estimatedEPS,surprise,surprisePercentage,ebit,ebitda,debt_to_equity,current_ratio,quick_ratio,return_on_equity
date,,,,,,,,,,
2024-03-31,2.94,2.82,0.12,4.2553,2.752700e+10,2.938000e+10,0.912981,1.241763,1.230761,0.086663
2023-12-31,2.93,2.78,0.15,5.3957,2.743500e+10,2.914300e+10,0.974911,1.217963,1.204618,0.091787
2023-09-30,2.99,2.65,0.34,12.8302,2.780900e+10,2.882500e+10,1.019740,1.663456,1.639416,0.100995
2023-06-30,2.69,2.55,0.14,5.4902,2.520900e+10,2.616700e+10,0.997721,1.769167,1.745163,0.097375
2023-03-31,2.45,2.23,0.22,9.8655,2.316900e+10,2.412900e+10,0.952343,1.912558,1.878984,0.093994
2022-12-31,2.32,2.29,0.03,1.3100,2.082900e+10,2.191800e+10,0.990608,1.931313,1.894846,0.089687
2022-09-30,2.35,2.30,0.05,2.1739,2.207200e+10,2.289400e+10,1.072894,1.840186,1.791347,0.101149
2022-06-30,2.23,2.29,-0.06,-2.6201,2.098300e+10,2.178400e+10,1.190679,1.784607,1.745251,0.100515
2022-03-31,2.22,2.19,0.03,1.3699,2.069300e+10,2.148400e+10,1.115140,1.987655,1.945092,0.102674


In [61]:
#next, we want to compute price-book ratio and price-earnings ratio for the stock
#first, let's pull daily stock data for "ticker" using alpha_vantage's TimeSeries class
from alpha_vantage.timeseries import TimeSeries
ts = TimeSeries(key=my_key,output_format='pandas')

In [63]:
#get daily stock data for "ticker"
df_stock, _ = ts.get_daily(ticker,outputsize='full')
df_stock

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2024-08-12,407.060,408.76,404.2434,406.81,16762883.0
2024-08-09,404.030,408.05,402.2624,406.02,19276666.0
2024-08-08,402.440,405.86,399.9407,402.69,20203030.0
2024-08-07,408.635,410.08,397.4700,398.43,20650906.0
2024-08-06,400.000,405.67,398.5000,399.61,24946525.0
...,...,...,...,...,...
1999-11-05,91.810,92.87,90.5000,91.56,35083700.0
1999-11-04,92.310,92.75,90.3100,91.75,27119700.0
1999-11-03,92.940,93.50,91.5000,92.00,22258500.0


In [64]:
df_financials.to_csv('../data/msft_financials.csv')
df_stock.to_csv('../data/msft_stock.csv')
df_ratios.to_csv('../data/msft_ratios.csv')

In [52]:
print(2*df_financials['totalCurrentAssets'])

date
2024-03-31    147180000000147180000000
2023-12-31    147393000000147393000000
2023-09-30    207586000000207586000000
2023-06-30    184257000000184257000000
2023-03-31    163889000000163889000000
2022-12-31    157823000000157823000000
2022-09-30    160812000000160812000000
2022-06-30    169684000000169684000000
2022-03-31    153922000000153922000000
2021-12-31    174188000000174188000000
2021-09-30    174326000000174326000000
2021-06-30    184406000000184406000000
2021-03-31    165614000000165614000000
2020-12-31    173973000000173973000000
2020-09-30    177077000000177077000000
2020-06-30    181915000000181915000000
2020-03-31    170505000000170505000000
2019-12-31    167074000000167074000000
2019-09-30    165896000000165896000000
2019-06-30    175552000000175552000000
2019-03-31    159887000000159887000000
2018-12-31    156874000000156874000000
2018-09-30    164195000000164195000000
2018-06-30    169662000000169662000000
2018-03-31    156659000000156659000000
2017-12-31    167633

In [10]:
test = df.copy()
test_earnings = df_earnings.copy()
test_earnings.head()

,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage,reportTime
date,,,,,,
2024-06-30,2024-07-30,2.95,2.93,0.02,0.6826,post-market
2024-03-31,2024-04-25,2.94,2.82,0.12,4.2553,post-market
2023-12-31,2024-01-30,2.93,2.78,0.15,5.3957,post-market
2023-09-30,2023-10-24,2.99,2.65,0.34,12.8302,post-market
2023-06-30,2023-07-25,2.69,2.55,0.14,5.4902,post-market


In [144]:
#drop all duplicate columns
df_financials = df_financials.loc[:,~df_financials.columns.duplicated()]
df_financials

,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,interestExpense,...,cashflowFromFinancing,paymentsForRepurchaseOfCommonStock,paymentsForRepurchaseOfEquity,dividendPayout,proceedsFromIssuanceOfCommonStock,proceedsFromRepurchaseOfEquity,changeInCashAndCashEquivalents,reportedDate,reportedEPS,reportTime
date,,,,,,,,,,,,,,,,,,,,,
2024-06-30,USD,39678000000,85777000000,52419000000,46099000000,25352000000,6320000000,8006000000,14326000000,NaN,...,-36017000000,2.652200e+10,2.652200e+10,3.895000e+09,NaN,-26522000000,NaN,2024-08-01,1.4000,post-market
2024-03-31,USD,42271000000,90753000000,54950000000,48482000000,27900000000,6468000000,7903000000,14371000000,NaN,...,-30433000000,2.320500e+10,2.320500e+10,3.710000e+09,NaN,-23205000000,NaN,2024-05-02,1.5300,post-market
2023-12-31,USD,54855000000,119575000000,71506000000,64720000000,40373000000,6786000000,7696000000,14482000000,1.002000e+09,...,-30585000000,2.013900e+10,2.013900e+10,3.825000e+09,NaN,-20139000000,NaN,2024-02-01,2.1800,post-market
2023-09-30,USD,40427000000,88496000000,55222000000,49071000000,26969000000,6151000000,7307000000,13458000000,1.002000e+09,...,-23153000000,2.100300e+10,2.100300e+10,3.758000e+09,0.0,-21003000000,8.390000e+08,2023-11-02,1.4600,post-market
2023-06-30,USD,36413000000,80799000000,51357000000,45384000000,22998000000,5973000000,7442000000,13415000000,9.980000e+08,...,-24048000000,1.747800e+10,0.000000e+00,3.849000e+09,0.0,0,2.769000e+09,2023-08-03,1.2600,post-market
2023-03-31,USD,41976000000,94836000000,59061000000,52860000000,28318000000,6201000000,7457000000,13658000000,9.300000e+08,...,-25724000000,1.959400e+10,1.959400e+10,3.650000e+09,NaN,-19594000000,5.155000e+09,2023-05-04,1.5200,post-market
2022-12-31,USD,50332000000,116151000000,73429000000,66822000000,36016000000,6607000000,7709000000,14316000000,1.003000e+09,...,-35563000000,1.947500e+10,1.947500e+10,3.768000e+09,0.0,-19475000000,-3.003000e+09,2023-02-02,1.8800,post-market
2022-09-30,USD,38095000000,89319000000,58491000000,52051000000,24894000000,6440000000,6761000000,13201000000,8.270000e+08,...,-26794000000,2.442800e+10,2.442800e+10,3.703000e+09,NaN,-24428000000,-3.884000e+09,2022-10-27,1.2900,post-market
2022-06-30,USD,35885000000,82240000000,53086000000,47074000000,23076000000,6012000000,6797000000,12809000000,7.190000e+08,...,-27445000000,2.186500e+10,2.186500e+10,3.811000e+09,NaN,-21865000000,1.944200e+10,2022-07-28,1.2000,post-market


In [147]:
print(df_financials.head(1))

           reportedCurrency  grossProfit  totalRevenue  costOfRevenue  \
date                                                                    
2024-06-30              USD  39678000000   85777000000    52419000000   

            costofGoodsAndServicesSold  operatingIncome  \
date                                                      
2024-06-30                 46099000000      25352000000   

            sellingGeneralAndAdministrative  researchAndDevelopment  \
date                                                                  
2024-06-30                       6320000000              8006000000   

            operatingExpenses  interestExpense  ...  cashflowFromFinancing  \
date                                            ...                          
2024-06-30        14326000000              NaN  ...           -36017000000   

            paymentsForRepurchaseOfCommonStock  paymentsForRepurchaseOfEquity  \
date                                                                    

In [145]:
list(df_financials.columns)

['reportedCurrency',
 'grossProfit',
 'totalRevenue',
 'costOfRevenue',
 'costofGoodsAndServicesSold',
 'operatingIncome',
 'sellingGeneralAndAdministrative',
 'researchAndDevelopment',
 'operatingExpenses',
 'interestExpense',
 'otherNonOperatingIncome',
 'depreciationAndAmortization',
 'incomeBeforeTax',
 'incomeTaxExpense',
 'netIncomeFromContinuingOperations',
 'comprehensiveIncomeNetOfTax',
 'ebit',
 'ebitda',
 'netIncome',
 'totalAssets',
 'totalCurrentAssets',
 'cashAndCashEquivalentsAtCarryingValue',
 'cashAndShortTermInvestments',
 'inventory',
 'currentNetReceivables',
 'totalNonCurrentAssets',
 'propertyPlantEquipment',
 'accumulatedDepreciationAmortizationPPE',
 'longTermInvestments',
 'shortTermInvestments',
 'otherCurrentAssets',
 'otherNonCurrentAssets',
 'totalLiabilities',
 'totalCurrentLiabilities',
 'currentAccountsPayable',
 'totalNonCurrentLiabilities',
 'otherCurrentLiabilities',
 'otherNonCurrentLiabilities',
 'totalShareholderEquity',
 'retainedEarnings',
 'comm